#Fake News Classifier Using LSTM
Dataset: https://www.kaggle.com/c/fake-news/data#

###Loading Data From Kaggle

In [5]:
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!kaggle competitions download -c fake-news

 94% 35.0M/37.0M [00:00<00:00, 73.6MB/s]
100% 37.0M/37.0M [00:00<00:00, 83.5MB/s]
  0% 0.00/40.6k [00:00<?, ?B/s]
100% 40.6k/40.6k [00:00<00:00, 42.0MB/s]
 96% 9.00M/9.42M [00:00<00:00, 91.4MB/s]
100% 9.42M/9.42M [00:00<00:00, 86.9MB/s]


In [ ]:
!unzip train.csv.zip
!unzip test.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                


###Importing Data 

In [ ]:
import pandas as pd
data=pd.read_csv("train.csv")
data.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


###Removing NaN Values from the dataset

In [ ]:
data=data.dropna()
data.reset_index(inplace=True)

###Extracting Dependent and Independent variables from data

In [ ]:
X = data.drop("label",axis=1)
y = data["label"]

###Checking The Shape of X and y

In [ ]:
print(X.shape)
print(y.shape)

(18285, 5)
(18285,)


###Importing the required Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

###Preprocessing The Text 



In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
messages=X.copy()
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
  review = re.sub("[^a-zA-Z]"," ",messages['title'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]  # (word not in stopwords) and stem(word)
  review = ' '.join(review)
  corpus.append(review)

### One Hot Encoding

In [ ]:
vocab_size=5000
onehot_repr=[one_hot(words,vocab_size)for words in corpus] 


[[2302, 78, 2269, 3679, 1743, 2457, 2605, 2930, 300, 3137],
 [281, 4812, 1772, 2611, 4067, 1021, 1732],
 [333, 1695, 1303, 2580],
 [4265, 4449, 2702, 3523, 1062, 3856],
 [2387, 4067, 2984, 3638, 3258, 4372, 4067, 2157, 2466, 1230],
 [2343,
  1705,
  1567,
  14,
  2302,
  478,
  1794,
  2923,
  4846,
  4626,
  4465,
  2671,
  4922,
  155,
  1732],
 [1732, 378, 906, 284, 74, 927, 3428, 3017, 1528, 813, 3132],
 [2292, 1119, 4282, 4411, 3569, 4086, 478, 324, 1528, 813, 3132],
 [528, 2048, 961, 2162, 3005, 1658, 1815, 1056, 478, 2580],
 [4119, 4754, 1681, 1113, 4505, 3775, 4546, 2785],
 [4589, 3612, 1567, 3908, 4236, 4483, 3184, 4928, 2899, 4875, 3691],
 [3523, 1863, 1743, 1658, 478, 3569],
 [596, 246, 4236, 599, 4782, 4962, 3733, 3486, 1329],
 [4480, 2767, 98, 3832, 1964, 2006, 2271, 1528, 813, 3132],
 [3400, 1789, 3594, 3179, 3230, 1528, 813, 3132],
 [3892, 1362, 2320, 2793, 4191, 2170, 2317, 3951, 52, 3574],
 [2954, 2916, 4812],
 [998, 4156, 685, 3651, 478, 4586, 3997, 1732],
 [3219, 870

###Applying Pad Sequences To Make Sentence Length Equal

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2930  300 3137]
 [   0    0    0 ... 4067 1021 1732]
 [   0    0    0 ... 1695 1303 2580]
 ...
 [   0    0    0 ... 1528  813 3132]
 [   0    0    0 ... 4683 2974 1146]
 [   0    0    0 ... 3376 3154 2237]]


#Creating Model for Train Set

In [ ]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)


Epoch 1/10
192/192 [==============================] - 8s 16ms/step - loss: 0.3345 - accuracy: 0.8408 - val_loss: 0.1989 - val_accuracy: 0.9148
Epoch 2/10
192/192 [==============================] - 2s 12ms/step - loss: 0.1396 - accuracy: 0.9458 - val_loss: 0.2223 - val_accuracy: 0.9085
Epoch 3/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0907 - accuracy: 0.9678 - val_loss: 0.2167 - val_accuracy: 0.9190
Epoch 4/10
192/192 [==============================] - 2s 13ms/step - loss: 0.0540 - accuracy: 0.9806 - val_loss: 0.2611 - val_accuracy: 0.9127
Epoch 5/10
192/192 [==============================] - 2s 12ms/step - loss: 0.0328 - accuracy: 0.9889 - val_loss: 0.2780 - val_accuracy: 0.9094
Epoch 6/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0169 - accuracy: 0.9949 - val_loss: 0.4052 - val_accuracy: 0.9070
Epoch 7/10
192/192 [==============================] - 2s 13ms/step - loss: 0.0105 - accuracy: 0.9968 - val_loss: 0.5776 - val_accuracy: 0.8999

###Performance Metrics And Accuracy



In [ ]:
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3058,  361],
       [ 175, 2441]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)


0.9111847555923778

Creating Model for Training
